Because word embeddings are very computationally expensive to train, most ML practitioners will load a pre-trained set of embeddings.


For this notebook, the word embeddings used will be a 50-dimensional GloVe vectors to represent words.

In [26]:
# Load packages
import numpy as np
import tensorflow as tf

In [27]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [28]:
words, word_to_vec_map = read_glove_vecs("glove.6B.50d.txt")

In [29]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similarity between u and v.
    cos(u, v) = u * v / (||u|| * ||v||)

    Arguments:
    u, v -- embeddings representation of words

    Returns:
    cosine_similarity -- the cosine angle between the 2 vectors.
    """

    # Base case, u and v are the same
    if np.all(u == v):
        return 1

    dot = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)

    # Avoid division by 0
    if np.isclose(norm_u * norm_v, 0, atol=1e-32):
        return 0

    cosine_similarity = dot / (norm_u * norm_v)
    return cosine_similarity

In [30]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Function that finds a word for the task: a is to b as c is to _____.
    e_b - e_a ~ e_w - e_c

    Arguments:
    word_a, word_b, word_c -- input words for the phrase
    word_to_vec_map -- dictionary that maps a word to the corresponding embedded vectors.

    Returns:
    best_word -- best word to fit the analogy between a and b, for c
    """

    # Convert the words into lowercase.
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()

    # Get the embedded representation for each.
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]

    # Get all the words to iter
    words = word_to_vec_map.keys()

    # Variables to find the most similar one to analogy
    max_cosine_sim = -100
    best_word = None

    for word in words:
        # Skip for the same word
        if word == word_c:
            continue
    
        # Get embedded version
        e_word = word_to_vec_map[word]

        # Check for similarity
        sim = cosine_similarity(e_b - e_a, e_word - e_c)

        if sim > max_cosine_sim:
            max_cosine_sim = sim
            best_word = word

    return best_word

In [31]:
# Test analogy
print("Man is to woman, as boy is to: " + complete_analogy("Man", "woman", "boy", word_to_vec_map))
print("Man is to programmer, as woman is to: " + complete_analogy("Man", "programmer", "woman", word_to_vec_map))

Man is to woman, as boy is to: girl
Man is to programmer, as woman is to: programmer


In [32]:
# Let's compute the vector g that roughly encodes the concept of "gender".
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


Let's use the similarity cosine function to see which names are closer to our female gender.

In [33]:
print ('List of names and their similarities with constructed vector:')

# Girls and boys names.
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

List of names and their similarities with constructed vector:
john -0.23163356145973732
marie 0.31559793539607295
sophie 0.3186878985941879
ronaldo -0.31244796850329437
priya 0.17632041839009405
rahul -0.16915471039231728
danielle 0.24393299216283904
reza -0.07930429672199556
katy 0.2831068659572616
yasmin 0.23313857767928767


Feminine names are closer to out "female gender" vector, which is expected.
Let's print the same computation but now using jobs.

In [34]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick 0.27691916256382676
guns -0.18884855678988982
science -0.060829065409297015
arts 0.008189312385880351
literature 0.06472504433459933
warrior -0.20920164641125286
doctor 0.11895289410935046
tree -0.0708939917547809
receptionist 0.3307794175059374
technology -0.13193732447554302
fashion 0.035638946257727
teacher 0.17920923431825672
engineer -0.08039280494524072
pilot 0.0010764498991917212
computer -0.10330358873850498
singer 0.18500518136496294


Some words are closer to female gender then male and vice-versa. 
It is astonishing how these results reflect certain unhealthy gender stereotypes.
For example, we see “computer” is negative and is closer in value to male first names, while “literature” is positive and is closer to female first names. Ouch!

You'll see below how to reduce the bias of these vectors, using an algorithm due to `Paperworks/Debiasing-word-embeddings.pdf`.
Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender-specific,
while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related.

In [35]:
# The paper assumes all word vectors to have L2 norm as 1 and hence the need for this calculation
from tqdm import tqdm
word_to_vec_map_unit_vectors = {
    word: embedding / np.linalg.norm(embedding)
    for word, embedding in tqdm(word_to_vec_map.items())
}
g_unit = word_to_vec_map_unit_vectors['woman'] - word_to_vec_map_unit_vectors['man']

100%|██████████| 400000/400000 [00:00<00:00, 453093.06it/s]


We can reduse bias by fixing a bias and a non-bias axis and project the non-gender words onto non-bias axis.
e_bias_component = (e * g / (||g|| ^ 2)) * g -- projection onto bias axis.
e_dibiased = e - e_bias_component

In [36]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    
    # Select word vector representation of "word".
    e = word_to_vec_map[word]
    
    # Compute e_biascomponent using the formula given above.
    e_biascomponent = np.dot(e, g) / (np.linalg.norm(g) ** 2) * g
 
    # Neutralize e by subtracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection.
    e_debiased = e - word_to_vec_map[word]
    
    return e_debiased

Test the debiasing function.

In [37]:
word = "receptionist"
print("cosine similarity between " + word + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map[word], g))

e_debiased = neutralize(word, g_unit, word_to_vec_map_unit_vectors)
print("cosine similarity between " + word + " and g_unit, after neutralizing: ", cosine_similarity(e_debiased, g_unit))

cosine similarity between receptionist and g, before neutralizing:  0.3307794175059374
cosine similarity between receptionist and g_unit, after neutralizing:  0


Next, let's see how debiasing can also be applied to word pairs such as "actress" and "actor".
Equalization is applied to pairs of words that you might want to have differ only through the gender property.
As a concrete example, suppose that "actress" is closer to "babysit" than "actor".
By applying neutralization to "babysit," you can reduce the gender stereotype associated with babysitting.
But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this.

In [38]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    # Select word vector representation of "word".
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    # Compute the mean of e_w1 and e_w2.
    mu = 1 / 2 * (e_w1 + e_w2)

    # Compute the projections of mu over the bias axis and the orthogonal axis.
    mu_B = np.dot(mu, bias_axis) / (np.linalg.norm(bias_axis) ** 2) * bias_axis
    mu_orth = mu - mu_B

    # Compute e_w1B and e_w2B.
    e_w1B = np.dot(e_w1, bias_axis) / (np.linalg.norm(bias_axis) ** 2) * bias_axis
    e_w2B = np.dot(e_w2, bias_axis) / (np.linalg.norm(bias_axis) ** 2) * bias_axis
        
    # Adjust the Bias part of e_w1B and e_w2B.
    corrected_e_w1B = np.sqrt(1 - np.linalg.norm(mu) ** 2) * (e_w1B - mu_B) / np.linalg.norm(e_w1B - mu_B)
    corrected_e_w2B = np.sqrt(1 - np.linalg.norm(mu) ** 2) * (e_w2B - mu_B) / np.linalg.norm(e_w2B - mu_B)

    # Debias by equalizing e1 and e2 to the sum of their corrected projections
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
                                                                
    return e1, e2

In [39]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g_unit, word_to_vec_map_unit_vectors)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g_unit))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g_unit))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.11711095765336835
cosine_similarity(word_to_vec_map["woman"], gender) =  0.35666618846270376

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.2387113614288377
cosine_similarity(e2, gender) =  0.2387113614288377
